In [6]:
# 필요한 라이브러리만 임포트
import numpy as np  # 다차원 배열을 다루기 위한 라이브러리
import librosa  # 음악 및 오디오 분석을 위한 라이브러리
import os  # 운영 체제와 상호 작용하기 위한 라이브러리, 파일 경로 조작 등
import tensorflow as tf  # 머신러닝 및 신경망을 위한 라이브러리
import pandas as pd  # 데이터 분석 및 조작을 위한 라이브러리
from IPython.display import clear_output  # Jupyter 노트북의 출력을 지우기 위한 함수
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 라이브러리

# Jupyter 노트북에서 matplotlib의 그래프를 인라인으로 표시하도록 설정
%matplotlib inline  

import seaborn as sns  # matplotlib 기반의 고급 시각화 라이브러리
import torch  # 딥러닝 프레임워크 중 하나
from sklearn.model_selection import train_test_split  # 데이터를 훈련 세트와 테스트 세트로 분할하기 위한 함수
from sklearn.metrics import accuracy_score, precision_score, recall_score  # 모델 평가를 위한 성능 지표 계산 함수
from tensorflow.keras import layers, losses  # TensorFlow의 케라스 API를 사용하여 신경망의 층과 손실 함수를 정의
from tensorflow.keras.models import Model  # TensorFlow의 케라스 API를 사용하여 모델을 정의 및 관리
from tensorflow.python.ops.numpy_ops import np_config  # TensorFlow에서 numpy와의 호환성을 위한 설정
np_config.enable_numpy_behavior()  # TensorFlow에서 numpy와 유사한 동작을 활성화

# 사용자 정의 유틸리티 함수 임포트
from util import play_audio, load_audio, get_features, get_mfcc, get_lmfe, get_chroma, plot_chroma, plot_mfcc, plot_lmfe
# 오디오 데이터를 다루기 위한 사용자 정의 함수들 (재생, 로드, 특징 추출 등)

# Seaborn 스타일 설정
sns.set(style="white", palette="muted")  # Seaborn 그래프의 기본 스타일 설정
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})  # Seaborn 그래프의 세부 스타일 설정

2024-06-18 15:59:42.004798: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 15:59:42.085337: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-18 15:59:42.085379: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [12]:
import json
from collections import Counter

def summarize_json(data, indent=0, key_prefix=""):
    """
    JSON 데이터를 요약하여 구조를 출력하는 함수
    """
    if isinstance(data, dict):
        print(' ' * indent + '{')
        for key, value in data.items():
            print(' ' * (indent + 2) + f'"{key}": ', end='')
            summarize_json(value, indent + 2, key_prefix=key_prefix + key + "/")
        print(' ' * indent + '}')
    elif isinstance(data, list):
        print(' ' * indent + '[')
        if data:
            item_types = Counter(type(item).__name__ for item in data)
            for item_type, count in item_types.items():
                print(' ' * (indent + 2) + f'{item_type} x {count}')
            for item in data[:1]:  # First item example
                summarize_json(item, indent + 2, key_prefix=key_prefix)
            if len(data) > 1:
                print(' ' * (indent + 2) + '...')
        print(' ' * indent + ']')
    else:
        print(' ' * indent + f'{type(data).__name__}')


위에 안 돌리고 prepare_dataset 을 테스트해보자

In [1]:
from python.prepare_dataset import get_data_paths_and_labels_from_machine as get_data_from_machine

In [2]:
from python.prepare_dataset import get_from_mimii
# "/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_04/normal/"

In [3]:
# base_dir = "/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/"

In [8]:
fan_data_path, fan_data_label = get_data_from_machine("fan")

base_dir is initialized to /mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data
['data', 'DCASE', '2020', 'dev', 'fan', 'train']
['data', 'DCASE', '2020', 'dev', 'fan', 'test']
['data', 'DCASE', '2020', 'eval', 'fan', 'test']
['data', 'DCASE', '2020', 'add', 'fan', 'train']
['data', 'DCASE', '2021', 'dev', 'fan', 'train']
['data', 'DCASE', '2021', 'dev', 'fan', 'test']
['data', 'DCASE', '2021', 'eval', 'fan', 'test']
['data', 'DCASE', '2021', 'add', 'fan', 'train']
['data', 'DCASE', '2022', 'dev', 'fan', 'train']
['data', 'DCASE', '2022', 'dev', 'fan', 'test']
['data', 'DCASE', '2022', 'eval', 'fan', 'test']
['data', 'DCASE', '2022', 'add', 'fan', 'train']
['data', 'DCASE', '2023', 'dev', 'fan', 'train']
['data', 'DCASE', '2023', 'dev', 'fan', 'test']
['data', 'DCASE', '2024', 'dev', 'fan', 'train']
['data', 'DCASE', '2024', 'dev', 'fan', 'test']
['data', 'MIMII', 'data_-6_db', 'fan', 'id_00', 'normal']
['data', 'MIMII', 'data_-6_db', 'fan', 'id_00', 'abnormal']
['data', 'M

In [17]:
print([].append([1, 2, 3]))

None


In [24]:
specific_fan_data = fan_data_path["MIMII"]["data_-6_db"]
specific_fan_data_list = list(specific_fan_data.values())[0] + list(specific_fan_data.values())[1]

In [29]:
print(len(list(specific_fan_data.values())[0]))
print(len(list(specific_fan_data.values())[1]))

1016
361


In [26]:
len(specific_fan_data_list)

1377

In [31]:
specific_fan_data_list[:10]

['/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/normal/00000305.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/normal/00000611.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/normal/00000000.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/normal/00000001.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/normal/00000002.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/normal/00000003.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/normal/00000004.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/normal/00000005.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII

In [32]:
specific_fan_data_list[1100:1110]

['/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/abnormal/00000305.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/abnormal/00000323.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/abnormal/00000341.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/abnormal/00000072.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/abnormal/00000073.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/abnormal/00000074.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/abnormal/00000075.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_-6_db/fan/id_06/abnormal/00000076.wav',
 '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/note

In [5]:
df_fan = pd.DataFrame({"filename" : fan_data_path["MIMII"]["data_-6_db"][""]

SyntaxError: incomplete input (2050613669.py, line 1)